In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import os
import time

gpu = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu[0], True)

In [2]:
IMAGE_SHAPE=(224, 224)
BATCH_SIZE=32
NUM_CLASSES = 5

In [3]:
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255, 
    width_shift_range=.1,
    height_shift_range=.1,
    zoom_range=0.05,
    channel_shift_range=0.05,
    validation_split=0.025)
train_data = image_generator.flow_from_directory(
    "/home/gragundier/Data/ffxiv_bot",
    target_size=IMAGE_SHAPE,
    subset="training",
    class_mode="categorical"
    )
validation_data = image_generator.flow_from_directory(
    "/home/gragundier/Data/ffxiv_bot", 
    target_size=IMAGE_SHAPE,
    subset="validation",
    class_mode="categorical"
    )

Found 6151 images belonging to 5 classes.
Found 155 images belonging to 5 classes.


In [4]:
'''

model = tf.keras.models.Sequential([
    # This is the first convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(6, activation='softmax')
])


model.summary()
'''

"\n\nmodel = tf.keras.models.Sequential([\n    # This is the first convolution\n    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(224, 224, 3)),\n    tf.keras.layers.MaxPooling2D(2, 2),\n    # The second convolution\n    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),\n    tf.keras.layers.MaxPooling2D(2,2),\n    # The third convolution\n    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),\n    tf.keras.layers.MaxPooling2D(2,2),\n    # The fourth convolution\n    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),\n    tf.keras.layers.MaxPooling2D(2,2),\n    # Flatten the results to feed into a DNN\n    tf.keras.layers.Flatten(),\n    tf.keras.layers.Dropout(0.5),\n    # 512 neuron hidden layer\n    tf.keras.layers.Dense(512, activation='relu'),\n    tf.keras.layers.Dense(6, activation='softmax')\n])\n\n\nmodel.summary()\n"

In [11]:
model = tf.keras.models.Sequential([
    # This is the first convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.2),
    # The second convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.2),
    # The third convolution
    #tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
    #tf.keras.layers.MaxPooling2D(2,2),
    #tf.keras.layers.Dropout(0.2),
    # The final convolution (same to make compute of reshape easier)
    tf.keras.layers.Conv2D(256, (3,3), padding='same', activation='relu'),
    #tf.keras.layers.MaxPooling2D(2,2),
    #tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    
    # Reshape to fit into LSTM/GRU
    tf.keras.layers.Reshape((54*54,256)),
    tf.keras.layers.GRU(15, activation='tanh', use_bias = True, recurrent_dropout=0),
    
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.2),
    # 512 neuron hidden layer
    #tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
    ])

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 222, 222, 128)     3584      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 111, 111, 128)     0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 111, 111, 128)     0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 109, 109, 128)     147584    
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 54, 54, 128)       0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 54, 54, 128)       0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 54, 54, 256)      

In [13]:
model.compile(loss = 'categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

In [ ]:
model_name = "cnn_mk4"
t = time.time()
log_dir = "/home/gragundier/Data/logs/{}".format("{}-{}".format(model_name, t))
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

history = model.fit(train_data, epochs=500, validation_data = validation_data, callbacks=[tensorboard_callback], verbose = 1)
#history = model.fit(train_data, epochs=500, validation_data = validation_data, verbose = 1)

Epoch 1/500
    775/Unknown - 164s 212ms/step - loss: 1.2468 - accuracy: 0.4968

In [19]:
#t = time.time()

export_path = "/tmp/saved_models/{}".format(int(t))
model.save(export_path, save_format='tf')

export_path

INFO:tensorflow:Assets written to: /tmp/saved_models/1586918879/assets


'/tmp/saved_models/1586918879'